# Telco Churn – Model Monitoring and Drift Analysis (Simulated)

This notebook is the **third part** of the Telco churn project.

Focus:
1. Train a solid churn model (Random Forest) on the Telco dataset.
2. **Simulate production monitoring** by splitting the test set into time windows.
3. Track metrics over time: accuracy, ROC-AUC, churn rate, and predicted positive rate.
4. Compute a simple **drift indicator** (Population Stability Index, PSI) for model scores.
5. Discuss how these ideas translate into a real production setup.

> This notebook is self-contained: it reloads and preprocesses the data from scratch.


## 1. Imports and Configuration

We import:

- `pandas`, `numpy` for data handling.
- `matplotlib`, `seaborn` for visualization.
- `scikit-learn` for preprocessing, modelling, and metrics.

We also set a fixed random seed for reproducibility.


In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

RANDOM_STATE: int = 42
np.random.seed(RANDOM_STATE)

DATA_PATH: Path = Path("data") / "WA_Fn-UseC_-Telco-Customer-Churn.csv"

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Data file not found at {DATA_PATH.resolve()}. "
        "Please download the Telco churn CSV from Kaggle and place it under the 'data/' directory."
    )


## 2. Data Loading and Cleaning

We repeat the same cleaning steps from the first notebook:

1. Load the CSV.
2. Convert `TotalCharges` to numeric.
3. Drop rows with missing `TotalCharges`.
4. Drop duplicate `customerID` entries.

This gives us a clean snapshot of Telco customers.


In [ ]:
def load_telco_data(path: Path) -> pd.DataFrame:
    """Load the Telco churn data from CSV.

    Args:
        path: Path to the CSV file.

    Returns:
        DataFrame with raw Telco data.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the loaded DataFrame is empty.
    """
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path!s}")

    df: pd.DataFrame = pd.read_csv(path)
    if df.empty:
        raise ValueError(f"Loaded DataFrame is empty: {path!s}")
    return df


def clean_telco_data(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Clean Telco data (types, missing values, duplicates).

    Args:
        raw_df: Raw Telco churn DataFrame.

    Returns:
        Cleaned DataFrame.
    """
    df = raw_df.copy()

    # Ensure the expected column exists
    if "TotalCharges" not in df.columns:
        raise ValueError("Expected 'TotalCharges' column not found.")

    # Convert TotalCharges to numeric, coercing errors to NaN
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

    # Show missing values
    missing = df.isna().sum()
    print("Missing values per column (non-zero only):")
    display(missing[missing > 0])

    # Drop rows with missing TotalCharges
    before = df.shape[0]
    df = df.dropna(subset=["TotalCharges"])
    after = df.shape[0]
    print(f"Dropped {before - after} rows with missing TotalCharges.")

    # Drop duplicate customers
    before = df.shape[0]
    df = df.drop_duplicates(subset=["customerID"])
    after = df.shape[0]
    print(f"Dropped {before - after} duplicate customerID rows.")

    df = df.reset_index(drop=True)
    return df


raw_df = load_telco_data(DATA_PATH)
telco_df = clean_telco_data(raw_df)
display(telco_df.head())


### Section summary

We loaded and cleaned the Telco churn dataset. The resulting DataFrame contains
one row per customer with consistent numeric and categorical features.

Next, we train a reference churn model that we will later monitor.


## 3. Train a Reference Churn Model

We:

1. Map the target `Churn` to 0/1.
2. Drop `customerID` from the features.
3. Split into train and test sets (stratified).
4. Build a preprocessing + model pipeline:
   - Scale numeric features.
   - One-hot encode categorical features.
   - Train a `RandomForestClassifier` with reasonable defaults.

This model acts as our **production model** for the rest of the notebook.


In [ ]:
TARGET_COL: str = "Churn"

if TARGET_COL not in telco_df.columns:
    raise KeyError(f"Target column {TARGET_COL!r} not found.")

X: pd.DataFrame = telco_df.drop(columns=[TARGET_COL, "customerID"])
y: pd.Series = telco_df[TARGET_COL].map({"No": 0, "Yes": 1})

categorical_cols: List[str] = [c for c in X.columns if X[c].dtype == "O"]
numeric_cols: List[str] = [c for c in X.columns if c not in categorical_cols]

print("Categorical columns:", categorical_cols)
print("Numeric columns:", numeric_cols)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE,
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)

rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        (
            "clf",
            RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_split=4,
                min_samples_leaf=2,
                random_state=RANDOM_STATE,
                n_jobs=-1,
            ),
        ),
    ]
)


In [ ]:
def evaluate_classifier(
    name: str,
    model: BaseEstimator,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
) -> Dict[str, float]:
    """Fit a classifier and compute basic metrics on train and test.

    Args:
        name: Model name (for printing).
        model: Unfitted sklearn estimator or pipeline.
        X_train: Training features.
        X_test: Test features.
        y_train: Training labels (0/1).
        y_test: Test labels (0/1).

    Returns:
        Dict with accuracy and ROC-AUC on the test set.
    """
    print(f"\n===== {name} =====")
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_proba_test = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_proba_test)
    else:
        y_proba_test = None
        roc_auc = np.nan

    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)

    print(f"Train accuracy: {acc_train:.3f}")
    print(f"Test accuracy:  {acc_test:.3f}")
    if not np.isnan(roc_auc):
        print(f"Test ROC-AUC:  {roc_auc:.3f}")

    print("\nClassification report (test):")
    print(classification_report(y_test, y_pred_test, target_names=["No churn", "Churn"]))

    cm = confusion_matrix(y_test, y_pred_test)
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=["Pred No", "Pred Yes"],
        yticklabels=["True No", "True Yes"],
    )
    plt.title(f"Confusion matrix - {name}")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

    return {
        "model": name,
        "train_accuracy": acc_train,
        "test_accuracy": acc_test,
        "roc_auc": float(roc_auc) if not np.isnan(roc_auc) else np.nan,
    }


baseline_metrics = evaluate_classifier(
    "Random Forest (reference)", rf_pipeline, X_train, X_test, y_train, y_test
)
baseline_metrics


### Section summary

We trained a **Random Forest churn model** and checked that its performance on
the test set is reasonable (decent ROC-AUC and recall for churners).

Next, we will treat this model as if it were deployed and examine how to
**monitor** it over time, using the test set as simulated production traffic.


## 4. Simulating Production Monitoring Windows

Real monitoring is usually **time-based** (daily, weekly, monthly). The Telco
dataset has no explicit timestamp, so we simulate time windows by:

1. Shuffling the test set (fixed seed).
2. Splitting it into consecutive **batches** (e.g. 10 windows).
3. Computing metrics per batch:
   - Accuracy
   - ROC-AUC
   - Churn rate in the data
   - Predicted positive rate

This lets us build **metrics over time**, as if each batch were a new day/week.


In [ ]:
def create_monitoring_windows(
    X: pd.DataFrame,
    y: pd.Series,
    n_windows: int = 10,
    random_state: int = 42,
) -> List[Dict[str, pd.DataFrame]]:
    """Split X and y into simulated time windows.

    Args:
        X: Feature DataFrame (test set).
        y: Target Series (test set).
        n_windows: Number of windows to create.
        random_state: Seed for shuffling.

    Returns:
        List of dicts with 'X' and 'y' for each window.
    """
    if len(X) != len(y):
        raise ValueError("X and y must have the same length.")

    # Shuffle indices to simulate random arrival
    rng = np.random.default_rng(random_state)
    indices = np.arange(len(X))
    rng.shuffle(indices)

    X_shuffled = X.iloc[indices].reset_index(drop=True)
    y_shuffled = y.iloc[indices].reset_index(drop=True)

    window_size: int = max(1, len(X) // n_windows)
    windows: List[Dict[str, pd.DataFrame]] = []

    for i in range(n_windows):
        start = i * window_size
        end = (i + 1) * window_size if i < n_windows - 1 else len(X_shuffled)

        if start >= len(X_shuffled):
            break

        X_win = X_shuffled.iloc[start:end].reset_index(drop=True)
        y_win = y_shuffled.iloc[start:end].reset_index(drop=True)

        windows.append({"X": X_win, "y": y_win})

    return windows


# Fit the RF pipeline once on the full training data
rf_pipeline.fit(X_train, y_train)

# Create simulated monitoring windows on the test set
monitoring_windows = create_monitoring_windows(X_test, y_test, n_windows=10, random_state=RANDOM_STATE)
len(monitoring_windows), monitoring_windows[0]["X"].shape


In [ ]:
def compute_window_metrics(
    model: BaseEstimator,
    window_idx: int,
    X_win: pd.DataFrame,
    y_win: pd.Series,
) -> Dict[str, float]:
    """Compute metrics for a single monitoring window.

    Args:
        model: Fitted sklearn estimator or pipeline.
        window_idx: Index of the window (0-based).
        X_win: Features in the window.
        y_win: Targets in the window.

    Returns:
        Dict with metrics and window index.
    """
    if len(X_win) == 0:
        raise ValueError("Window is empty; cannot compute metrics.")

    y_pred = model.predict(X_win)

    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_win)[:, 1]
        roc_auc = roc_auc_score(y_win, y_proba)
    else:
        y_proba = None
        roc_auc = np.nan

    accuracy = accuracy_score(y_win, y_pred)
    churn_rate = float(y_win.mean())  # proportion of 1s
    positive_rate = float(y_pred.mean())  # proportion predicted as churn

    return {
        "window": float(window_idx),
        "n_samples": float(len(y_win)),
        "accuracy": float(accuracy),
        "roc_auc": float(roc_auc) if not np.isnan(roc_auc) else np.nan,
        "churn_rate": float(churn_rate),
        "predicted_positive_rate": float(positive_rate),
    }


window_metrics: List[Dict[str, float]] = []

for idx, win in enumerate(monitoring_windows):
    metrics = compute_window_metrics(
        model=rf_pipeline,
        window_idx=idx,
        X_win=win["X"],
        y_win=win["y"],
    )
    window_metrics.append(metrics)

metrics_df = pd.DataFrame(window_metrics)
display(metrics_df)


In [ ]:
# Plot metrics over windows
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(metrics_df["window"], metrics_df["accuracy"], marker="o")
axes[0, 0].set_title("Accuracy over windows")
axes[0, 0].set_xlabel("Window")
axes[0, 0].set_ylabel("Accuracy")

axes[0, 1].plot(metrics_df["window"], metrics_df["roc_auc"], marker="o")
axes[0, 1].set_title("ROC-AUC over windows")
axes[0, 1].set_xlabel("Window")
axes[0, 1].set_ylabel("ROC-AUC")

axes[1, 0].plot(metrics_df["window"], metrics_df["churn_rate"], marker="o")
axes[1, 0].set_title("Observed churn rate over windows")
axes[1, 0].set_xlabel("Window")
axes[1, 0].set_ylabel("Churn rate")

axes[1, 1].plot(metrics_df["window"], metrics_df["predicted_positive_rate"], marker="o")
axes[1, 1].set_title("Predicted positive rate over windows")
axes[1, 1].set_xlabel("Window")
axes[1, 1].set_ylabel("Predicted positive rate")

plt.tight_layout()
plt.show()


### Section summary

We simulated **monitoring windows** and computed key metrics in each one:

- Performance metrics: accuracy, ROC-AUC.
- Data properties: observed churn rate.
- Model behaviour: predicted positive rate.

In a real system, you would log these metrics continuously and display them on
a dashboard (e.g. Grafana, Superset, internal BI tools). Thresholds or control
limits could trigger alerts when metrics move outside acceptable ranges.

Next we add a simple **drift measure** on model scores using Population
Stability Index (PSI).


## 5. Score Distribution Drift with PSI (Population Stability Index)

One common way to monitor model drift is to compare the distribution of:

- Features, or
- Model scores (predicted probabilities)

between a **reference period** (training data) and a **current period** (a
monitoring window).

Here we:

1. Use the **train** score distribution as the reference.
2. For each window, compute the PSI between the train scores and the window scores.
3. Plot PSI over windows.

Rule-of-thumb for PSI:

- `< 0.1` – No significant drift.
- `0.1–0.25` – Moderate drift; watch closely.
- `> 0.25` – Large drift; investigate and possibly retrain.


In [ ]:
def calculate_psi(
    expected: np.ndarray,
    actual: np.ndarray,
    n_bins: int = 10,
) -> float:
    """Calculate the Population Stability Index (PSI) between two score arrays.

    Args:
        expected: Reference scores (e.g. train predictions).
        actual: Current scores (e.g. window predictions).
        n_bins: Number of bins for the score histogram.

    Returns:
        PSI value (higher means more drift).
    """
    if len(expected) == 0 or len(actual) == 0:
        raise ValueError("Expected and actual arrays must be non-empty.")

    # Define common bin edges from the expected distribution
    quantiles = np.linspace(0.0, 1.0, n_bins + 1)
    bin_edges = np.quantile(expected, quantiles)

    # Small epsilon to avoid division by zero or log of zero
    eps = 1e-6

    # Compute histogram for expected and actual
    expected_counts, _ = np.histogram(expected, bins=bin_edges)
    actual_counts, _ = np.histogram(actual, bins=bin_edges)

    expected_perc = expected_counts / max(expected_counts.sum(), eps)
    actual_perc = actual_counts / max(actual_counts.sum(), eps)

    psi_values = (actual_perc - expected_perc) * np.log(
        (actual_perc + eps) / (expected_perc + eps)
    )

    return float(np.sum(psi_values))


# Score distributions: train vs each window
train_scores = rf_pipeline.predict_proba(X_train)[:, 1]

psi_records: List[Dict[str, float]] = []

for idx, win in enumerate(monitoring_windows):
    window_scores = rf_pipeline.predict_proba(win["X"])[:, 1]
    psi_value = calculate_psi(expected=train_scores, actual=window_scores, n_bins=10)

    psi_records.append({"window": float(idx), "psi": float(psi_value)})

psi_df = pd.DataFrame(psi_records)
display(psi_df)


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(psi_df["window"], psi_df["psi"], marker="o")
plt.axhline(0.1, linestyle="--")  # moderate drift threshold
plt.axhline(0.25, linestyle="--")  # high drift threshold
plt.title("Score distribution drift (PSI) over windows")
plt.xlabel("Window")
plt.ylabel("PSI")
plt.show()


### Section summary

We computed **PSI** between:

- The training score distribution (reference), and
- Each monitoring window score distribution (current).

Then we plotted PSI over windows, with simple thresholds at 0.1 and 0.25.

In real deployments, you might:

- Compute PSI for multiple features and the score distribution.
- Trigger alerts when PSI exceeds thresholds for several consecutive periods.
- Investigate data-quality issues, feature distribution changes, or shifts in
  the underlying population.


## 6. Simple Alert Logic (Example)

To make the idea concrete, we can implement a basic **alert rule**:

- If PSI > 0.25 in any window, print a **high drift** warning.
- If ROC-AUC in a window falls below a chosen threshold (e.g. 0.70),
  print a **performance degradation** warning.

This is just a toy example; real systems would integrate with logging,
monitoring, and alerting infrastructure.


In [ ]:
def check_alerts(
    metrics_df: pd.DataFrame,
    psi_df: pd.DataFrame,
    min_roc_auc: float = 0.70,
    max_psi: float = 0.25,
) -> None:
    """Check simple alert conditions based on ROC-AUC and PSI.

    Args:
        metrics_df: DataFrame with window-level metrics (including 'window' and 'roc_auc').
        psi_df: DataFrame with window-level PSI values (columns 'window' and 'psi').
        min_roc_auc: Minimum acceptable ROC-AUC before raising an alert.
        max_psi: Maximum acceptable PSI before raising an alert.
    """
    # Merge metrics and PSI by window
    merged = metrics_df.merge(psi_df, on="window", how="inner")

    for _, row in merged.iterrows():
        w = int(row["window"])
        roc_auc = float(row["roc_auc"])
        psi = float(row["psi"])

        if roc_auc < min_roc_auc:
            print(f"[ALERT] Window {w}: ROC-AUC {roc_auc:.3f} < {min_roc_auc:.2f} (performance degradation).")

        if psi > max_psi:
            print(f"[ALERT] Window {w}: PSI {psi:.3f} > {max_psi:.2f} (significant drift).")


check_alerts(metrics_df=metrics_df, psi_df=psi_df, min_roc_auc=0.70, max_psi=0.25)


### Section summary

We implemented a minimal **alerting layer** on top of our monitoring metrics.

In practice, this logic would be integrated with:

- Dashboards (Grafana, Looker, internal tools).
- Alerting systems (Slack, email, PagerDuty, etc.).
- A retraining / revalidation pipeline when alerts persist.

The key idea is that **monitoring is not only about a single accuracy number**;
you track both **performance metrics** and **data/score drift** to decide when
to investigate or intervene.


## 7. Final Thoughts and Next Steps

Across the three notebooks you now have a complete mini-project:

1. **Notebook 1 – Core churn project**
   - Business framing, data understanding, cleaning, EDA.
   - Baseline and main models.
   - Feature importance and interpretation.

2. **Notebook 2 – Tuning and cost-sensitive decisions**
   - Hyperparameter tuning with cross-validation.
   - Cost-aware threshold selection.
   - Precision–recall and threshold analysis.

3. **Notebook 3 – Monitoring and drift (this one)**
   - Simulated production windows.
   - Metrics over time (accuracy, ROC-AUC, churn and prediction rates).
   - Score drift monitoring with PSI.
   - Simple alert rules.

Possible extensions:

- Use real timestamps (if available) instead of simulated windows.
- Persist monitoring data to a database or data lake and build dashboards on top.
- Add more advanced drift detection methods (e.g. statistical tests, domain-specific checks).
- Integrate monitoring and retraining into an automated MLOps pipeline.

This trilogy is a solid starting point for a **churn modelling workflow** that
is not just about building a model, but also about **operating it responsibly
over time**.
